# Data Mining Project
by Obsa Aba-waji Mirnary Carbajal

## Experiment 1: Linear Regression
In this notebook, we will be looking at a comparison of gas prices with fuel economy. Our hypothesis is that increases in gas prices are what drive the increase in improvements of fuel economy.

In [1]:
%matplotlib inline

## Importing Libs and Data

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
vehicles = pd.read_csv('data/vehicles.csv', dtype=object)
weekly_gas_all_prices = pd.read_csv('data/GASALLCOVW.csv', index_col=0)
weekly_gas_reg_prices = pd.read_csv('data/GASREGCOVW.csv', index_col=0)
weekly_gas_prem_prices = pd.read_csv('data/GASPRMCOVW.csv', index_col=0)
annually_gas_all_prices = pd.read_csv('data/GASALLCOVA.csv', index_col=0)
annually_gas_reg_prices = pd.read_csv('data/GASREGCOVA.csv', index_col=0)
annually_gas_prem_prices = pd.read_csv('data/GASPRMCOVA.csv', index_col=0)

## Check to see that Vehicles Data has been loaded

Looking at the head of the data to make sure that the data has been loaded into their dataframes.

In [4]:
vehicles.head()

,barrels08,barrelsA08,charge120,charge240,city08,city08U,cityA08,cityA08U,cityCD,cityE,...,mfrCode,c240Dscr,charge240b,c240bDscr,createdOn,modifiedOn,startStop,phevCity,phevHwy,phevComb
0,15.695714285714287,0.0,0.0,0.0,19,0.0,0,0.0,0.0,0.0,...,NaN,NaN,0.0,NaN,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0
1,29.96454545454546,0.0,0.0,0.0,9,0.0,0,0.0,0.0,0.0,...,NaN,NaN,0.0,NaN,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0
2,12.207777777777778,0.0,0.0,0.0,23,0.0,0,0.0,0.0,0.0,...,NaN,NaN,0.0,NaN,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0
3,29.96454545454546,0.0,0.0,0.0,10,0.0,0,0.0,0.0,0.0,...,NaN,NaN,0.0,NaN,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0
4,17.347894736842107,0.0,0.0,0.0,17,0.0,0,0.0,0.0,0.0,...,NaN,NaN,0.0,NaN,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0


## Cleaning up vehicles data

The vehicles data has a lot of fields that we will not be needing and is not ogranized in a logical manner for our needs.

In [5]:
columns_for_vehicles = ['year',
                        'make',
                        'model',
                        'VClass',
                        'fuelType',
                        'fuelType1',
                        'city08',
                        'highway08',
                        'comb08']
vehicles = vehicles[columns_for_vehicles]
vehicles.head()

,year,make,model,VClass,fuelType,fuelType1,city08,highway08,comb08
0,1985,Alfa Romeo,Spider Veloce 2000,Two Seaters,Regular,Regular Gasoline,19,25,21
1,1985,Ferrari,Testarossa,Two Seaters,Regular,Regular Gasoline,9,14,11
2,1985,Dodge,Charger,Subcompact Cars,Regular,Regular Gasoline,23,33,27
3,1985,Dodge,B150/B250 Wagon 2WD,Vans,Regular,Regular Gasoline,10,12,11
4,1993,Subaru,Legacy AWD Turbo,Compact Cars,Premium,Premium Gasoline,17,23,19


In [6]:
categorical_columns = ['make', 'VClass', 'fuelType', 'fuelType1']
numeric_columns = ['city08', 'highway08', 'comb08']

In [7]:
for col in vehicles.columns:
    if col in categorical_columns:
        vehicles[col] = pd.Categorical(vehicles[col])
        vehicles['{col}_codes'.format(col=col)] = vehicles[col].cat.codes
    elif col in numeric_columns:
        vehicles[col] = pd.to_numeric(vehicles[col])

In [8]:
vehicles.head()

,year,make,model,VClass,fuelType,fuelType1,city08,highway08,comb08,make_codes,VClass_codes,fuelType_codes,fuelType1_codes
0,1985,Alfa Romeo,Spider Veloce 2000,Two Seaters,Regular,Regular Gasoline,19,25,21,3,29,11,5
1,1985,Ferrari,Testarossa,Two Seaters,Regular,Regular Gasoline,9,14,11,38,29,11,5
2,1985,Dodge,Charger,Subcompact Cars,Regular,Regular Gasoline,23,33,27,31,28,11,5
3,1985,Dodge,B150/B250 Wagon 2WD,Vans,Regular,Regular Gasoline,10,12,11,31,30,11,5
4,1993,Subaru,Legacy AWD Turbo,Compact Cars,Premium,Premium Gasoline,17,23,19,117,0,7,4


## Cleaning up Gasoline data

The next piece of data we need to clean up is our gas prices data. Our vehicle data has years prior to 1993, but the data is not complete, so we will be focusing on gas prices starting in 1994.

In [9]:
annually_gas_prem_prices['prem_velocity'] = annually_gas_prem_prices.diff()
annually_gas_all_prices['all_velocity'] = annually_gas_all_prices.diff()
annually_gas_reg_prices['reg_velocity'] = annually_gas_reg_prices.diff()

In [10]:
temp = annually_gas_all_prices.join(annually_gas_prem_prices)
joined_annually_gas_prices = temp.join(annually_gas_reg_prices)
len(joined_annually_gas_prices)

22

In [11]:
joined_annually_gas_prices.head()

,GASALLCOVA,all_velocity,GASPRMCOVA,prem_velocity,GASREGCOVA,reg_velocity
DATE,,,,,,
1995-01-01,1.144,NaN,1.287,NaN,1.103,0.031
1996-01-01,1.231,0.087,1.371,0.084,1.192,0.089
1997-01-01,1.228,-0.003,1.368,-0.003,1.189,-0.003
1998-01-01,1.056,-0.172,1.199,-0.169,1.017,-0.172
1999-01-01,1.156,0.100,1.297,0.098,1.116,0.099


## Experiment

Now that the data has been cleaned, we can focus on the first experiment. In this experiment, we will be looking at the speed changes of gas prices over the years, then comparing their effect to the effect of increases in fuel effiency. 

### Splitting the data
The first step is going to be splitting out data into testing and training data as well as into input data vs output data

In [12]:
#X = fruits[['height', 'width', 'mass', 'color_score']]
#y = fruits['fruit_label']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

NameError: name 'train_test_split' is not defined